# Lec27 `#19/12`

## Compilation

### ec-eval -> compile-eval

**Basic Idea**: Saving the executions that would have been runned by ec-eval to store them, however this happens to be slow (as expected), and it needs to be optimized

*Same register set*

### Watching ec-eval of (f x) :

```scheme
; Prepare to eval operator (save env, operands, continue)
(save continue)
(save env)
(assign unev (op operands) (reg exp))
(save unev)
; Figure out what operator is
(assign exp (op operands) (reg exp))
(assign continue (label ev-appl-did-operator))
(goto (label eval-dispatch))
(test (op self-evaluating?) (reg exp))
(branch (label ev-self-eval))
(test (op variable?) (reg exp))
; It's a variable, so look up
(branch (label ev-variable))
(assign val (op lookup-variable-value) (reg exp) (reg env))
(goto (reg continue))
; ev-appl-did-operator
(restore unev)
(restore env)
(assign argl (op empty-arglist))
; Store operator in proc
(assign proc (reg val))
(test (op no-operands?) (reg unev))
; Branch is never taken
(branch (label apply-dispatch))
; Eval each of the operands
(save proc)
(save argl)
(assign exp (op first-operand) (reg unev))
(test (op) (last-operand?) (reg unev))
(branch (label ev-appl-last-arg))
(assign continue (label ev-appl-accum-last-arg))
; Figure out how to eval first operand
(goto (label eval-dispatch))
(test (op self-evaluating?) (reg exp))
(branch (label ev-self-eval))
(test (op variable?) (reg exp))
(branch (label ev-variable))
(assign val (op lookup-variable-value) (reg exp) (reg env))
; Goto accum-last-arg
(goto (reg continue))
; Add to arg list
(restore argl)
(assign argl (op adjoin-arg) (reg val) (reg argl))
(restore proc)
; Have proc and argl, ready to apply
; Computation proceeds at apply-dispatch
```

Then the unnecessary operations are removed (iteratively): 

```scheme
; Prepare to eval operator (save env, operands, continue)
;; (save continue)
;; (save env)
;; (assign unev (op operands) (reg exp))
;; (save unev)
; Figure out what operator is
;; (assign exp (op operands) (reg exp))
#|
(assign continue (label ev-appl-did-operator))
(goto (label eval-dispatch))
(test (op self-evaluating?) (reg exp))
(branch (label ev-self-eval))
(test (op variable?) (reg exp))
; It's a variable, so look up
(branch (label ev-variable))
(assign val (op lookup-variable-value) (reg exp) (reg env))
|#
; Change to const f
;;(assign val (op lookup-variable-value) (const f) (reg env))
; Assign to proc directly
(assign proc (op lookup-variable-value) (const f) (reg env))
;; (goto (reg continue))
; ev-appl-did-operator
;; (restore unev)
;; (restore env)
;; (assign argl (op empty-arglist))
; Store operator in proc
;; (assign proc (reg val))
;; (test (op no-operands?) (reg unev))
; Branch is never taken
;; (branch (label apply-dispatch))
; Eval each of the operands
;; (save proc)
;; (save argl)
;; (assign exp (op first-operand) (reg unev))
#|
(test (op) (last-operand?) (reg unev))
(branch (label ev-appl-last-arg))
(assign continue (label ev-appl-accum-last-arg))
; Figure out how to eval first operand
(goto (label eval-dispatch))
(test (op self-evaluating?) (reg exp))
(branch (label ev-self-eval))
(test (op variable?) (reg exp))
(branch (label ev-variable))
(assign val (op lookup-variable-value) (reg exp) (reg env))
|#
(assign val (op lookup-variable-value) (const x) (reg env))
; Goto accum-last-arg
;; (goto (reg continue))
; Add to arg list
;; (restore argl)
;; (assign argl (op adjoin-arg) (reg val) (reg argl))
(assign argl (op list) (reg val))
;; (restore proc)
; Have proc and argl, ready to apply
; Computation proceeds at apply-dispatch


```

`(f x)`, optimized as much as possible:

```scheme
; The above call
(assign proc (op lookup-variable-value) (const f) (reg env))
(assign val (op lookup-variable-value) (const x) (reg env))
(assign argl (op list) (reg val))
; Then <apply-dispatch> (optimized):
(test (op primitive-procedure?) (reg proc))
(branch (label primivite-branch9))
compiled-branch8:
(assign continue (label after-call7))
(assign val (op compiled-procedure-entry) (reg proc))
(goto (reg val))
primitive-branch9:
(assign val (op primitive-procedure-entry) (reg proc))
after-call7:
```